# FFT Python Serial

- https://numpy.org/doc/stable/reference/routines.fft.html
- https://hgomersall.github.io/pyFFTW/

In [1]:
%%writefile pc2cs.py
import numpy as np, pyfftw as pf, time as tm

t0 = tm.time()    # time measurement

# data
L = M = N = 576
u = pf.empty_aligned( (N, M, L), dtype=np.complex128 )
for k in range (u.shape[2]) :
    for j in range(u.shape[1]) :
        for i in range(u.shape[0]) :
            u[i, j, k] = i + j + k + 3
u.real = np.sin ( u.real )
u.imag = 0

t1 = tm.time()    # time measurement

# FFT
uf = pf.interfaces.numpy_fft.fftn(u,
        overwrite_input=True, auto_contiguous=False,
        auto_align_input=False)
# checksum
S = np.sum(uf)

t2 = tm.time()    # time measurement

print(f"S: {S*1E-5:.0f}", end='')
print(f" | L: {L:0g}", end='')
print(f" | T1: {t1-t0:.4f}", end='')
print(f" | TF: {t2-t1:.4f}", end='')
print(f" | TT: {t2-t0:.4f}")

Overwriting pc2cs.py


## Copy to /scratch

In [3]:
%%bash
dst=/scratch${PWD#"/prj"}
cp pc2cs.py $dst

## Batch script

In [4]:
%%writefile pc2cs.srm
#!/bin/bash
#SBATCH --job-name pc2cs       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:15:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
cd
dir=/scratch${PWD#"/prj"}
cd $dir
source $dir/env2/etc/profile.d/conda.sh
conda activate $dir/env2
conda activate --stack $dir/env3
cd $dir/fft
                                              
# Executable
EXEC="python pc2cs.py"

# Start
echo '-- run --------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting pc2cs.srm


<hr style="height:10px;border-width:0;background-color:green">

## Run

In [5]:
%%bash
sbatch pc2cs.srm
sbatch pc2cs.srm
sbatch pc2cs.srm

Submitted batch job 1331134
Submitted batch job 1331135
Submitted batch job 1331136


In [6]:
! squeue -n pc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1331134  cpu_small  PD  0:00     1    1
           1331135  cpu_small  PD  0:00     1    1
           1331136  cpu_small  PD  0:00     1    1


In [2]:
%%bash
cat /scratch${PWD#"/prj"}/slurm-1331134.out
cat /scratch${PWD#"/prj"}/slurm-1331135.out
cat /scratch${PWD#"/prj"}/slurm-1331136.out

- Job ID: 1331134
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1498
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python pc2cs.py
-- output -----------------------------
S: 270-0j | L: 576 | T1: 154.7522 | TF: 18.7891 | TT: 173.5413
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331135
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1450
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python pc2cs.py
-- output -----------------------------
S: 270-0j | L: 576 | T1: 148.0629 | TF: 18.7748 | TT: 166.8378
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331136
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes 

<hr style="height:10px;border-width:0;background-color:red">